<a href="https://colab.research.google.com/github/talktokorea/storage/blob/master/fishy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 연동
#구글 드라이브를 코랩에 마운트 Mount Google Drive
from google.colab import drive     # import drive from google colab
ROOT="/content/drive"                # root directory
drive.mount(ROOT)                     # mount google drive at ROOT= /content/drive
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/storage"
%cd {PROJECT_PATH}
%pwd


# matplotlib 사용/버전 확인    https://matplotlib.org/stable/users/installing/index.html
import matplotlib                      # matplotlib을 import
matplotlib.__version__             # version


In [ ]:
# 0. 판다스 라이브러리 임포트
import  pandas  as  pd

# 1. github 데이터 읽기
fish = pd.read_csv("https://raw.githubusercontent.com/talktokorea/storage/master/fish.csv")

# 2. 데이터 모양/속성 확인      (49,3)
print(fish.shape)      
#fish.tail()

# 3. 컬럼명 확인 Index(['length', 'weight', 'species'], dtype='object')
print(fish.columns)

# 4. species의 유일한 값 확인  ['Bream', 'Swelt’] 
print(pd.unique(fish['species']))

# 5. 입력 fish_input 생성   학습 및 테스트 데이터 분리까지 배열을 사용하지 않음(ANN 에서 활용 염두) 
#fish_input = fish[ ['length', 'weight'] ].to_numpy()
fish_input = fish[ ['length', 'weight'] ]
print(fish_input[:5])

# 6. 타겟 fish_target 생성
fish_target = fish[ ['species'] ].to_numpy()
#fish_target = fish[ ['species'] ]
print(fish_target[:5])

In [ ]:
# 7. plot 생성
import matplotlib.pyplot as plt
fish_plt=fish_input.to_numpy()
plt.scatter(fish_plt[:,0],fish_plt[:,1])
plt.xlabel('length')
plt.ylabel('weight')
plt.show()


In [ ]:
# 8. fish_input과 fish_target을 훈련(train) 세트/테스트(test) 세트 구분(25%)
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(fish_input, fish_target, stratify=fish_target, random_state=42)
print(train_input)
print(test_input)
print(train_target)
print(test_target)

# 9. 표준화 전처리 #  파일 형태의 train 및 test 데이터가 numpy 배열로 변경됨 
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_input_scaled = ss.transform(train_input)
test_input_scaled = ss.transform(test_input)

print(train_input_scaled)
print(test_input_scaled)

In [ ]:
# 10. 학습
from sklearn.neighbors  import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
#kn.fit(train_input_scaled, train_target)
kn.fit(train_input_scaled, train_target.ravel())
# 11. 정확도
print(kn.score(train_input_scaled,train_target))
print(kn.score(test_input_scaled,test_target))
# 12. 클래스 이름 순서(알파벳순)  ['Bream','Smelt']
print(kn.classes_)
# 13. test_input_scaled의 최초 5개에 대한 예측
print(test_input[:5])
print(test_input_scaled[:5])
print(kn.predict(test_input_scaled[:5]))

# 14. 소수점 3자리 반올림 확률
import numpy as np
proba = kn.predict_proba(test_input_scaled[:5])
print(np.round(proba,decimals=3))

# 15. 최근접 이웃 클래스 확인
distances, indexes = kn.kneighbors(test_input_scaled[3:4])
print(train_target[indexes])
print(distances)

In [ ]:
# 16. confusion matrix, classication_report
from sklearn.metrics import confusion_matrix, classification_report
test_predict=kn.predict(test_input_scaled[:])
confusion_matrix(test_target,test_predict)
#print("hello")
classification_report(test_target,test_predict)

In [ ]:
# 17. 새로운 생선[25,150]에 대한 예측
from sklearn.preprocessing import StandardScaler
# 신규 입력을 data frame으로 설정. 이유는 이전에 표준화를 df를 대상으로 했기 때문에. 
# 즉, ss.transform(train_input)의 train_input이 현재 df임
# df_new = [[25,150]]  또는 np.array([[25,150]])으로 설정하면 값은 계산되지만 warning이 발생함
df_new = pd.DataFrame( [[25,150]], columns=['length','weight'] )
ss = StandardScaler()
# 평균, 표준편차 계산
ss.fit(train_input)
# z 값 계산
new_scaled = ss.transform(df_new)

# 18. new_scaled에 대한 예측
print(df_new)
print(new_scaled)
print(kn.predict(new_scaled))


In [ ]:
# 19. plotting
import matplotlib.pyplot as plt
distances, indexes = kn.kneighbors(new_scaled)
plt.scatter(new_scaled[:,0],new_scaled[:,1],marker='^')
plt.scatter(train_input_scaled[:,0],train_input_scaled[:,1])
plt.scatter(train_input_scaled[indexes,0],train_input_scaled[indexes,1],marker='D')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()


